<h1> Machine Learning w chmurze - Cloud ML </h1>

W tym notebooku pokażemy jak przenieść prosty model Tensorflow do GCP i uruchomić na nim predykcje

<h2> Predykcje na podstawie tekstu </h2>

<b>Źródło danych</b>: Yelp Restaurant Reviews (https://www.yelp.com/dataset/challenge)

Dataset zawiera między innymi informacje o restauracjach oraz opinie klientów

Zadaniem jest przewidzenie czy restauracje przejdą inspekcje (amerykańskiego) sanepidu na podstawie opinii gości oraz dodatkowych informacji takich jak lokalizacja i rodzaje kuchni serwowanych w restauracji.

<h2> Ustawienie zmiennych środowiskowych, import bibliotek </h2>

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
BUCKET = 'dswbiznesie'
PROJECT = 'dswbiznesie'
REGION = 'europe-west1'

In [11]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [12]:
%datalab project set -p $PROJECT

In [26]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

In [16]:
import google.datalab.ml as ml
import tensorflow as tf
import apache_beam as beam
import shutil
import datetime
from apache_beam.io.gcp.internal.clients import bigquery
print tf.__version__

1.2.1


<h2> Dane źródłowe </h2>


Dataset pobrany ze strony Yelp zawiera następujące pliki

In [65]:
!gsutil -q ls -l gs://$BUCKET/rawdata/hygiene

 103859986  2017-11-02T23:29:21Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat
    762559  2017-11-02T23:29:22Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat.additional
     90363  2017-11-02T23:29:23Z  gs://dswbiznesie/rawdata/hygiene/hygiene.dat.labels
TOTAL: 3 objects, 104712908 bytes (99.86 MiB)


* <b>hygiene.dat</b>: każda linia zawiera połączone opinie klientów danej restauracji
* <b>hygiene.dat.labels</b>: dla pierwszych 546 linii przypisana jest dodatkowe pole w którym 0 oznacza to że restauracja przeszła inspekcje, 1 to że restauracja <b>nie</b> przeszła inspekcji. Reszta wpisów posiada wpis "[None]" co oznacza że należą do zbioru testowego
* <b>hygiene.dat.additional</b>: plik CSV gdzie w pierwszym polu znajduje się lista oferowanych rodzajów kuchnii, w drugim kod pocztowy restauracji który można uznać za przybliżenie lokalizacji restauracji. W trzecim polu znajduje się liczba opinii, w czwartym średnia ocena ( w skali 0-5, gdzie 5 oznacza ocene najlepszą).

<h2> Feature engineering używając Apache Beam i BigQuery</h2>

Dane źródłowe należy przetowrzyć i dostosować do postaci której będzie można łatwo użyć do uczenia i ewaluacji modelu. 
Najwygodniejszym choć nie najtańszym rozwiązaniem jest załadowanie danych do BigQuery.

Odpowiednim narzędziem do tego zadania jest Apache Beam i jego implementacja - Google Dataflow.
Job Dataflow uruchamiany jest w chmurze. Jego przebieg można śledzić w Konsoli GCP (https://console.cloud.google.com/dataflow).
Uruchomienie joba trwa powyżej minuty.

In [76]:
#clear BigQuery table here

In [33]:
def create_record(rest_tuple):
    #print(rest_tuple)
    identity = rest_tuple[0]
    reviews = rest_tuple[1]['reviews_kv'][0]
    inspection_result = int(rest_tuple[1]['labels_kv'][0]) if rest_tuple[1]['labels_kv'][0] != "[None]" else None
    categories_temp = rest_tuple[1]['attributes_kv'][0].split("\"")
    categories = [ x.replace('\'', '').replace('[','').replace(']','') for x in categories_temp[1].split(',')]
    attributes_temp = categories_temp[2].split(",")
    zip_code = attributes_temp[1]
    review_count = int(attributes_temp[2])
    avg_rating = float(attributes_temp[3])
    
    return { 
        'identity': identity, 
        'reviews': reviews,
        'inspection_result': inspection_result,
        'categories': categories,
        'zip_code': zip_code,
        'review_count': review_count,
        'avg_rating': avg_rating
    }

def preprocess(RUNNER,BUCKET,BIGQUERY_TABLE):
    job_name = 'hygiene-ftng' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S')
    print 'Launching Dataflow job {} ... hang on'.format(job_name)
    OUTPUT_DIR = 'gs://{0}/data/hygiene/'.format(BUCKET)
    options = {
        'staging_location': os.path.join(OUTPUT_DIR, 'tmp', 'staging'),
        'temp_location': os.path.join(OUTPUT_DIR, 'tmp'),
        'job_name': 'hygiene-ftng' + '-' + datetime.datetime.now().strftime('%y%m%d-%H%M%S'),
        'project': PROJECT,
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True
    }
    opts = beam.pipeline.PipelineOptions(flags=[], **options)
    p = beam.Pipeline(RUNNER, options=opts)
    
    # Adding table definition
    table_schema = bigquery.TableSchema()
    
    # Fields definition
    identity_schema = bigquery.TableFieldSchema()
    identity_schema.name = 'identity'
    identity_schema.type = 'integer'
    identity_schema.mode = 'required'
    table_schema.fields.append(identity_schema)
    
    
    reviews_schema = bigquery.TableFieldSchema()
    reviews_schema.name = 'reviews'
    reviews_schema.type = 'string'
    reviews_schema.mode = 'required'
    table_schema.fields.append(reviews_schema)

    inspection_result_schema = bigquery.TableFieldSchema()
    inspection_result_schema.name = 'inspection_result'
    inspection_result_schema.type = 'integer'
    inspection_result_schema.mode = 'nullable'
    table_schema.fields.append(inspection_result_schema)
    
    categories_schema = bigquery.TableFieldSchema()
    categories_schema.name = 'categories'
    categories_schema.type = 'string'
    categories_schema.mode = 'repeated'
    table_schema.fields.append(categories_schema)
    
    zip_code_schema = bigquery.TableFieldSchema()
    zip_code_schema.name = 'zip_code'
    zip_code_schema.type = 'string'
    zip_code_schema.mode = 'required'
    table_schema.fields.append(zip_code_schema)
    
    review_count_schema = bigquery.TableFieldSchema()
    review_count_schema.name = 'review_count'
    review_count_schema.type = 'integer'
    review_count_schema.mode = 'required'
    table_schema.fields.append(review_count_schema)
    
    avg_rating_schema = bigquery.TableFieldSchema()
    avg_rating_schema.name = 'avg_rating'
    avg_rating_schema.type = 'float'
    avg_rating_schema.mode = 'required'
    table_schema.fields.append(avg_rating_schema)
    
    #processing logic
    reviews = p | 'readreviews' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat'.format(BUCKET))
    labels = p | 'readlabels' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat.labels'.format(BUCKET))
    attributes = p | 'readattributes' >> beam.io.ReadFromText('gs://{0}/rawdata/hygiene/hygiene.dat.additional'.format(BUCKET))
    
    reviews_kv = reviews | 'mapreviews to kv' >> beam.Map(lambda x: (x.split(",")[0], x))
    labels_kv = labels | 'maplabels to kv' >> beam.Map(lambda x: (x.split(",")[0], x.split(",")[1]))
    attributes_kv = attributes | 'mapattributes to kv' >> beam.Map(lambda x: (x.split(",")[0], x))
    
    restaurants = (
        {'reviews_kv': reviews_kv, 'labels_kv': labels_kv, 'attributes_kv': attributes_kv}
        | 'CoGroupByRestaurantKey' >> beam.CoGroupByKey())
    
    records = restaurants | 'CreateRecords' >> beam.Map(create_record)
    
    records | 'write' >> beam.io.Write(
        beam.io.BigQuerySink(
            BIGQUERY_TABLE,
            schema=table_schema,
            create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
            write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE))
    p.run().wait_until_finish()
    print("Done!")

<b> Uruchomienie etl </b>

In [34]:
bigquery_dataset = PROJECT+":"+PROJECT+".hygiene"
preprocess('DirectRunner',BUCKET, bigquery_dataset)

Launching Dataflow job hygiene-ftng-171106-014621 ... hang on
Done!


<h2> Model Tensorflow </h2>

<h2> Trenowanie modelu w Cloud ML </h2>

<h2> Serwowanie predykcji </h2>